In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
%load_ext autoreload
%autoreload 2

import pickle
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from spectral_adversarial_regularization.models import elunet as model
from spectral_adversarial_regularization import dl_utils

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Xtr = mnist.train.images.reshape(-1, 28, 28, 1)
Ytr = mnist.train.labels.astype(float)

Xtt = mnist.test.images.reshape(-1, 28, 28, 1)
Ytt = mnist.test.labels.astype(float)

num_classes = 10
val_set = {'X': Xtt[:500], 'Y': Ytt[:500]}
C2 = np.mean([np.sqrt(np.sum(np.square(i))) for i in Xtr])
eps = 0.05*C2

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
def train_network(Xtr, Ytr, val_set, arch, save_dir, eps=0.3,
                  gpu_prop=0.05, num_epochs=100, save_every=25, beta=1,
                  step_adv=None, adv_robustness=None, retrain=True):
    
    if retrain: os.system('rm -rf %s'%(save_dir))
        
    _ = dl_utils.build_graph_and_train(Xtr, Ytr, save_dir, num_classes,
                                       arch=arch,
                                       num_epochs=num_epochs,
                                       save_every=save_every,
                                       num_channels=1,
                                       val_set=val_set,
                                       early_stop_acc=0.995,
                                       gpu_prop=gpu_prop,
                                       beta=beta,
                                       eps=eps,
                                       step_adv=step_adv,
                                       adv_robustness=adv_robustness)


from spectral_adversarial_regularization import adversarial as ad

def get_adv_acc_curve(X, Y, save_dir, arch, eps_list, order=2, method=ad.fgm, beta=1.):
    
    adv_accs = np.zeros(len(eps_list))
    acc = dl_utils.build_graph_and_predict(X, save_dir, arch=arch, Y=Y, beta=beta, num_channels=1)
    print('Acc on examples: %.2f'%(acc))
    for i, eps in enumerate(eps_list):
        adv_accs[i] = ad.test_net_against_adv_examples(X, Y, save_dir, arch, beta=beta, num_channels=1,
                                                       method=method, order=order, eps=eps)
    return acc, adv_accs

In [ ]:
arch = model.elunet
methods = ['wrm']

for method in methods:
    save_dir = os.path.join('save_weights_final', 'mnist', method)
    train_network(Xtr, Ytr, val_set, arch, save_dir, eps=eps, adv_robustness=method)

Epoch 14/100 (1825.769 s), batch 167/550 (85.434 s): loss 0.008, acc 1.0000

In [ ]:
# Evaluate robustness of networks
eps_list = np.arange(0, 2.5, 0.25)
methods = ['erm', 'fgm', 'pgm', 'wrm']
adv_results_pgm = {}

for method in methods:
    save_dir = os.path.join('save_weights_final', 'mnist', method)
    s_norm = dl_utils.get_overall_sn(num_classes, save_dir, arch, num_channels=1)
    acc, adv_accs = get_adv_acc_curve(Xtt[500:2500], Ytt[500:2500], save_dir, arch, eps_list,
                                            order=2, method=ad.pgm, beta=1.)
    
    adv_results_pgm[method] = (acc, adv_accs, s_norm)
    
plt.figure(figsize=(10, 7))
for method in methods:
    plt.plot(eps_list/C2, 1.-adv_results_pgm[method][1], label='%s (test acc %.2f, sn %.2e)'\
             %(method, adv_results_pgm[method][0], adv_results_pgm[method][2]))
    
plt.xlabel(r'$\epsilon/C_2$')
plt.ylabel('Error')
plt.ylim(1e-2, 1e0)
plt.yscale('log')
plt.title('PGM attacks')
plt.legend(bbox_to_anchor=(1.4, 1))
plt.grid()
plt.show()